<img align="right" width="300" src="https://data-services.hosting.nyu.edu/assets/libraries_short_color.png" alt="NYU Libraries Logo">

# Deidentification by Design: Creating Ethical Data Derivatives with Python

**Overview** 

Research and proprietary data often contain personally identifiable information, with variables that reveal details about the lives of individuals and may have been collected without the person’s knowledge or consent. Datasets aggregated at the individual level often interest social science scholars, yet such data poses a risk of identification and create an ethical dilemma for curators.
 
While some types of information and data are legally protected, other social data, such as home mortgage files, voter registration files, and tax parcel records are public and are often augmented with modeled indicators, such as religious belief or personal income, that may not represent the reality of people's lives.  Library information and data specialists must develop infrastructure, workflows, and policies to ensure the ethical stewardship and use of these datasets.  This interactive workshop will explore the tension between making purchased data as widely accessible to researchers as possible, while also ensuring that sensitive data is not abused. 
 
Following a short discussion of some of the above challenges, we will introduce technologies and workflows for data de-identification via hands-on activities in you will create redacted samples of data that maintain research integrity and usefulness. Learning outcomes include:
 
 - Develop fluency with generating random samples in order to make analysis with large files more manageable
 - Know how to assess the identification risk of specific variables within a dataset in order to protect the identity of human subjects
 - Create a Jupyter Notebook workflow that enables cleaning, redacting, and sharing data for research use
 - Learn some fundamental Pandas features for exploring, cleaning, and transforming data

**About the Sample Data:**

The L2 Political files come in two parts for each state: Demographic indicators and Voter History. Both are .tab files (tab delimited) and will need to be extracted. It is likely that you'll need to use a command line tool like The Unarchiver to decompress each archive. 

For larger states, you will want to redact parts of these files by only loading the minimum number of variables that you need into memory, merge them together, then sample them in order to perform analysis.

This notebook is designed to provide some guidance on working with the data and creating extracts in a memory-efficient manner. It uses Python Libraries that are included with the Anacondas distribution, with the exception of [Pyjanitor](https://pyjanitor.readthedocs.io/index.html). In order to install it, run

`conda install pyjanitor -c conda-forge`

The conda distribution here seems to work the best (as opposed to pip). See the documentation for each of these libraries (especially Pandas) for more information. You will need to restart your Kernel and notebook before proceeding if you haven't already installed Pyjanitor. 

Users are expected to have agreed to the Terms of Use and should create redacted versions of the data that strip out identifying information and load only variables that are important for analysis. For the latest documentation, visit the [L2 Political LibGuide](https://guides.nyu.edu/l2political)

## I. Loading Python modules and L2 Data

Begin by importing some basic libraries to work with the data 

In [ ]:
import pandas as pd
import janitor

**Loading .tab files**

First, we read in the .tab file of demographic variables, declare the tab as the separator, and set the encoding

There are 646 columns in the demographic file, and it's better to set the datetype as string (labeled "object" in Pandas) for each of them and only load certian columns to cut down on memory usage.

Declare the columns you want; see the codebook for the full list.


In [ ]:
selected_variables = ['LALVOTERID',
                      'Residence_Addresses_Latitude',
                      'Residence_Addresses_Longitude',
                      'Voters_Gender',
                      'Voters_FIPS']

## Change the file path below accordingly to point toward full path the relevant file. 
## Note that this data loading step below will take the longest out of any step for your computer to process
## The arguments that set the encoding, define the sep, load only some variables,
## and force all columns into a string are necessary
## to prevent Pandas from guessing the encoding or data types incorrectly (and wasting resources)

state_demographic = pd.read_csv('VM2--WY--2021-01-13-DEMOGRAPHIC.tab', 
                                sep='\t', dtype=str, encoding='unicode_escape',
                                usecols=selected_variables)

## Preview of the first 5 rows of the data subset. 

state_demographic.head(5)

It's also not a bad idea to test the length of the entire file and verify it with the number of records on the documentation

In [ ]:
len(state_demographic)

You can also run .info to get a breakdown, including how much memory the data frame is eating up.

This will give you context for how much your computer can handle.

In [ ]:
state_demographic.info()

Next, we will read into memory the voter history file that corresponds to the state.

This time, we make sure to include the *LALVOTERID* column, and we will add whether or not they voted in 2016.

In [ ]:
needed_variables = ['LALVOTERID', 'General_2016_11_08']

state_voterhistory = pd.read_csv('VM2--WY--2021-01-13-VOTEHISTORY.tab',
                                 sep='\t', dtype=str, encoding='unicode_escape',
                                 usecols=needed_variables)
                                
state_voterhistory.head(5)

In [ ]:
## Again, I will want to run an .info command to test the size and memory

state_voterhistory.info()

## II. Join of Demographic and Voter History Tables

Now that I have established a two reduced tables with a connection, I can join them (using the merge method in Pandas) based on the *LALVOTERID* common key


In [ ]:
merged_file = pd.merge(state_voterhistory, state_demographic,
                       how='left', left_on='LALVOTERID', right_on='LALVOTERID')

merged_file.head(5)

## III. Sampling the Records

From here, you can sample the records. The frac argument is in percentage, so here, it's a one percent sample note that the sample function also guesses the dtype (which is proccess intensive) so it's better to do the sample after the final merge

In [ ]:
state_sample = merged_file.sample(frac=0.01)

state_sample.head(5)

Again, print out the info to see that its a valid sample size (actually one percent) and that it is using much less memory

You'll know that the sampling worked if the index numbers are nonsequential and random also, you can test the length and info of your new sample

In [ ]:
state_sample.info()

## IV. Further Deidentifying Geocoded Locations: Jittering

There are many other filtering and cleaning steps that could be taken before writing out a new file.

For example, the coordinates can be identifying, even if you strip out all of the other PID variables, because they are geocoded to the address. You will want to jitter them if you intend to aggregate or visualize data.

First, print out the data types to make sure what they are:

In [ ]:
state_sample.dtypes

In [ ]:
## In order to jitter the coordinates, you need to change them to a float

state_sample['Residence_Addresses_Latitude'] = pd.to_numeric(state_sample['Residence_Addresses_Latitude']) 
state_sample['Residence_Addresses_Longitude'] = pd.to_numeric(state_sample['Residence_Addresses_Longitude']) 

state_sample.dtypes

The module pyjanitor introduces a jitter method for dataframes that can be used to displace the exact coordinates of a location.

We'll start with jittering the latitude. This library inserts a Gaussian distribution ("noise") equal to the standard deviation of the scale. The exact amount of the scale in jitter may need to change but this is a good starting place. You may want to test the results in a mapping software like QGIS or ArcMap before proceeding too far.

In [ ]:
jittered_lat = state_sample.jitter(
    column_name='Residence_Addresses_Latitude',
    dest_column_name='lat_jitter',
    scale=0.05,
    clip=None,
    random_state=None,
)

jittered_lat.head(5)

In [ ]:
## Now we jitter the longitude

jittered_long = state_sample.jitter(
    column_name='Residence_Addresses_Longitude',
    dest_column_name='long_jitter',
    scale=0.05,
    clip=None,
    random_state=None,
)

jittered_long.head(5)

To complete the jittering process, make sure you drop off the actual Residence_Addresses columns.

With this last dataframe, you should have a one percent sample that has jittered lat long coordinates, voter gender, whether or not they voted in 2016, and their unique ID:


In [ ]:
stripped_sample = jittered_long.drop(columns=['Residence_Addresses_Latitude', 'Residence_Addresses_Longitude'])

stripped_sample.head(5)

## V. Saving the Resulting Dataframe to a CSV

Finally, you may want to write out the sample to a CSV, leaving out the index that Pandas creates and stipulating the encoding into utf-8. Remember to set the full path on your local machine.

In [ ]:
stripped_sample.to_csv(r'wyoming-onepercent-sample.csv', index=False, encoding='utf-8')

In the course of your analysis, you may want to repeat this process for multiple states and/or develop an aggregate roster of voters culled from a subset. You are encouraged to visit the tools section on the [L2 Political LibGuide at NYU Libraries](https://guides.nyu.edu/l2political/tools).

In general, you should have a data management plan with these files that includes the following elements:

- Secure any original .tab files you've downloaded by storing them in a single directory location, and then deleting the original .tab files once you've generated extracts
- Manage your extracted samples in a secure environment

Simple scripts and suggestions are available on the L2 Political LibGuide. 
